This notebook will read/load the newspaper htmls, transform them and prepare the simple dimensions, which are prepared to match the following analyse. The analyse will be about the word "klima" and its changing use over time by newspaper. See Readme for more.

In [2]:
import os
import sys
import logging
import glob
sys.path.append(os.path.abspath("pylib"))

import pandas as pd
from handle_sqlite import save_dataframe_to_db, read_table_as_dataframe
from handle_data_processing import process_newspaper_with_context


# Load all the newspapers
Here we will load the csv for one day.

In [3]:
newspapers_test = [
    {"file_name": "data_input/data_lake/2021-04-02-54books.html", "encoding": "utf-8", "name": "54books", "date": "2021-04-02"},
    {"file_name": "data_input/data_lake/2021-04-02-abendblatt.html", "encoding": "utf-8", "name": "abendblatt", "date": "2021-04-02"},
    {"file_name": "data_input/data_lake/2025-01-14-vice-de.html", "encoding": "utf-8", "name": "vice", "date": "2025-01-24"},
    {"file_name": "data_input/data_lake/2021-04-24-tagesschau.html", "encoding": "utf-8", "name": "tagesschaus", "date": "2021-04-24"},
    {"file_name": "data_input/data_lake/2025-01-24-tagesschau.html", "encoding": "utf-8", "name": "tagesschaus", "date": "2025-01-24"},
]

In [4]:
# Use glob to list all CSV files in the specified directory with date format in their names
csv_files = glob.glob('data_input/data_lake/*-*.csv')

In [6]:
# Initialize an empty list to store newspaper data
newspapers = []

# Process each CSV file
for csv_file in csv_files:
    # Load CSV
    df = pd.read_csv(csv_file)

    # Filter by status 200 and select necessary columns
    valid_newspapers = df[df['status'] == 200][['name', 'date', 'file_name', 'encoding']]

    # Convert valid rows to dictionaries and add to newspapers list
    newspapers.extend(valid_newspapers.to_dict(orient='records'))

In [7]:
newspapers

[{'name': 'sz',
  'date': '2025-01-14',
  'file_name': 'data-lake/2025-01-14-sz.html',
  'encoding': 'utf-8'},
 {'name': 'zeit',
  'date': '2025-01-14',
  'file_name': 'data-lake/2025-01-14-zeit.html',
  'encoding': 'UTF-8'},
 {'name': 'faz',
  'date': '2025-01-14',
  'file_name': 'data-lake/2025-01-14-faz.html',
  'encoding': 'utf-8'},
 {'name': 'heise',
  'date': '2025-01-14',
  'file_name': 'data-lake/2025-01-14-heise.html',
  'encoding': 'utf-8'},
 {'name': 'golem',
  'date': '2025-01-14',
  'file_name': 'data-lake/2025-01-14-golem.html',
  'encoding': 'UTF-8'},
 {'name': 'tagesspiegel',
  'date': '2025-01-14',
  'file_name': 'data-lake/2025-01-14-tagesspiegel.html',
  'encoding': 'utf-8'},
 {'name': 'taz',
  'date': '2025-01-14',
  'file_name': 'data-lake/2025-01-14-taz.html',
  'encoding': 'utf-8'},
 {'name': 'abendblatt',
  'date': '2025-01-14',
  'file_name': 'data-lake/2025-01-14-abendblatt.html',
  'encoding': 'utf-8'},
 {'name': 'berliner',
  'date': '2025-01-14',
  'file_na

In [8]:
metadata_collection = []
context_collection = []

for newspaper in newspapers:
    try:
        metadata, context_data = process_newspaper_with_context(newspapers)
                
        # Add a unique ID for each newspaper in the metadata and add to context
        newspaper_id = len(metadata_collection) + 1  # This can be a simple counter for unique IDs (or use UUID)
        metadata["newspaper_id"] = newspaper_id  # Add newspaper_id to metadata
        
        # Append the metadata to its respective collection
        metadata_collection.append(metadata)
        
        # Append the context data with id to its respective collection if 'klima' was found at least once
        if metadata['klima_mentions_count'] > 0:
            # First add the same newspaper_id to each context data
            for context in context_data:
                context["newspaper_id"] = newspaper_id
            context_collection.extend(context_data) # Using extend here because context_data is already a list of dicts

    except Exception as e:
        logging.error(f"Error processing {newspaper['name']} for {newspaper['date']}: {e}")

2025-02-22 18:15:21,589 - ERROR - Error processing sz for 2025-01-14: list indices must be integers or slices, not str
2025-02-22 18:15:21,591 - ERROR - Error processing zeit for 2025-01-14: list indices must be integers or slices, not str
2025-02-22 18:15:21,592 - ERROR - Error processing faz for 2025-01-14: list indices must be integers or slices, not str
2025-02-22 18:15:21,593 - ERROR - Error processing heise for 2025-01-14: list indices must be integers or slices, not str
2025-02-22 18:15:21,593 - ERROR - Error processing golem for 2025-01-14: list indices must be integers or slices, not str
2025-02-22 18:15:21,594 - ERROR - Error processing tagesspiegel for 2025-01-14: list indices must be integers or slices, not str
2025-02-22 18:15:21,595 - ERROR - Error processing taz for 2025-01-14: list indices must be integers or slices, not str
2025-02-22 18:15:21,596 - ERROR - Error processing abendblatt for 2025-01-14: list indices must be integers or slices, not str
2025-02-22 18:15:21,

2025-02-22 18:15:21,637 - ERROR - Error processing handelsblatt for 2025-01-02: list indices must be integers or slices, not str
2025-02-22 18:15:21,638 - ERROR - Error processing ntv for 2025-01-02: list indices must be integers or slices, not str
2025-02-22 18:15:21,638 - ERROR - Error processing pioneer for 2025-01-02: list indices must be integers or slices, not str
2025-02-22 18:15:21,639 - ERROR - Error processing suedwest for 2025-01-02: list indices must be integers or slices, not str
2025-02-22 18:15:21,640 - ERROR - Error processing t3n for 2025-01-02: list indices must be integers or slices, not str
2025-02-22 18:15:21,640 - ERROR - Error processing economist for 2025-01-02: list indices must be integers or slices, not str
2025-02-22 18:15:21,641 - ERROR - Error processing srf for 2025-01-02: list indices must be integers or slices, not str
2025-02-22 18:15:21,642 - ERROR - Error processing wef for 2025-01-02: list indices must be integers or slices, not str
2025-02-22 18:15

2025-02-22 18:15:21,691 - ERROR - Error processing kindly for 2021-04-02: list indices must be integers or slices, not str
2025-02-22 18:15:21,691 - ERROR - Error processing stuttgarter for 2021-04-02: list indices must be integers or slices, not str
2025-02-22 18:15:21,692 - ERROR - Error processing atlantic for 2021-04-02: list indices must be integers or slices, not str
2025-02-22 18:15:21,693 - ERROR - Error processing netzpolitik for 2021-04-02: list indices must be integers or slices, not str
2025-02-22 18:15:21,694 - ERROR - Error processing towardsds for 2021-04-02: list indices must be integers or slices, not str
2025-02-22 18:15:21,695 - ERROR - Error processing uebermedien for 2021-04-02: list indices must be integers or slices, not str
2025-02-22 18:15:21,695 - ERROR - Error processing vulture for 2021-04-02: list indices must be integers or slices, not str
2025-02-22 18:15:21,696 - ERROR - Error processing boerse for 2021-04-02: list indices must be integers or slices, not

2025-02-22 18:15:21,738 - ERROR - Error processing anwaltsverein for 2023-07-14: list indices must be integers or slices, not str
2025-02-22 18:15:21,738 - ERROR - Error processing wiwo for 2023-07-14: list indices must be integers or slices, not str
2025-02-22 18:15:21,739 - ERROR - Error processing dw-en for 2023-07-14: list indices must be integers or slices, not str
2025-02-22 18:15:21,739 - ERROR - Error processing dav for 2023-07-14: list indices must be integers or slices, not str
2025-02-22 18:15:21,740 - ERROR - Error processing zwanzig for 2023-07-14: list indices must be integers or slices, not str
2025-02-22 18:15:21,741 - ERROR - Error processing blick for 2023-07-14: list indices must be integers or slices, not str
2025-02-22 18:15:21,742 - ERROR - Error processing nzz for 2023-07-14: list indices must be integers or slices, not str
2025-02-22 18:15:21,745 - ERROR - Error processing republik for 2023-07-14: list indices must be integers or slices, not str
2025-02-22 18:15

2025-02-22 18:15:21,788 - ERROR - Error processing ta for 2023-07-24: list indices must be integers or slices, not str
2025-02-22 18:15:21,788 - ERROR - Error processing watson-ch for 2023-07-24: list indices must be integers or slices, not str
2025-02-22 18:15:21,789 - ERROR - Error processing nau for 2023-07-24: list indices must be integers or slices, not str
2025-02-22 18:15:21,789 - ERROR - Error processing standard for 2023-07-24: list indices must be integers or slices, not str
2025-02-22 18:15:21,790 - ERROR - Error processing kronen for 2023-07-24: list indices must be integers or slices, not str
2025-02-22 18:15:21,790 - ERROR - Error processing kurier for 2023-07-24: list indices must be integers or slices, not str
2025-02-22 18:15:21,791 - ERROR - Error processing kleine for 2023-07-24: list indices must be integers or slices, not str
2025-02-22 18:15:21,792 - ERROR - Error processing watson-de for 2023-07-24: list indices must be integers or slices, not str
2025-02-22 18:1

2025-02-22 18:15:21,834 - ERROR - Error processing welt for 2025-01-24: list indices must be integers or slices, not str
2025-02-22 18:15:21,835 - ERROR - Error processing medium for 2025-01-24: list indices must be integers or slices, not str
2025-02-22 18:15:21,836 - ERROR - Error processing esslinger for 2025-01-24: list indices must be integers or slices, not str
2025-02-22 18:15:21,836 - ERROR - Error processing ntv for 2025-01-24: list indices must be integers or slices, not str
2025-02-22 18:15:21,836 - ERROR - Error processing pioneer for 2025-01-24: list indices must be integers or slices, not str
2025-02-22 18:15:21,837 - ERROR - Error processing suedwest for 2025-01-24: list indices must be integers or slices, not str
2025-02-22 18:15:21,838 - ERROR - Error processing t3n for 2025-01-24: list indices must be integers or slices, not str
2025-02-22 18:15:21,838 - ERROR - Error processing economist for 2025-01-24: list indices must be integers or slices, not str
2025-02-22 18:1

2025-02-22 18:15:21,887 - ERROR - Error processing stuttgarter for 2023-07-02: list indices must be integers or slices, not str
2025-02-22 18:15:21,888 - ERROR - Error processing atlantic for 2023-07-02: list indices must be integers or slices, not str
2025-02-22 18:15:21,888 - ERROR - Error processing netzpolitik for 2023-07-02: list indices must be integers or slices, not str
2025-02-22 18:15:21,889 - ERROR - Error processing towardsds for 2023-07-02: list indices must be integers or slices, not str
2025-02-22 18:15:21,890 - ERROR - Error processing uebermedien for 2023-07-02: list indices must be integers or slices, not str
2025-02-22 18:15:21,890 - ERROR - Error processing vulture for 2023-07-02: list indices must be integers or slices, not str
2025-02-22 18:15:21,891 - ERROR - Error processing boerse for 2023-07-02: list indices must be integers or slices, not str
2025-02-22 18:15:21,892 - ERROR - Error processing buchreport for 2023-07-02: list indices must be integers or slices,

2025-02-22 18:15:21,936 - ERROR - Error processing boerse for 2021-04-14: list indices must be integers or slices, not str
2025-02-22 18:15:21,936 - ERROR - Error processing buchreport for 2021-04-14: list indices must be integers or slices, not str
2025-02-22 18:15:21,937 - ERROR - Error processing 54books for 2021-04-14: list indices must be integers or slices, not str
2025-02-22 18:15:21,937 - ERROR - Error processing dlf for 2021-04-14: list indices must be integers or slices, not str
2025-02-22 18:15:21,938 - ERROR - Error processing dw for 2021-04-14: list indices must be integers or slices, not str
2025-02-22 18:15:21,939 - ERROR - Error processing spiegel for 2021-04-14: list indices must be integers or slices, not str
2025-02-22 18:15:21,939 - ERROR - Error processing cnn for 2021-04-14: list indices must be integers or slices, not str
2025-02-22 18:15:21,940 - ERROR - Error processing bbc for 2021-04-14: list indices must be integers or slices, not str
2025-02-22 18:15:21,941

In [18]:
# Convert to DataFrame after processing all newspapers
final_metadata_df = pd.DataFrame(metadata_collection)
final_context_df = pd.DataFrame(context_collection)

In [19]:
# Save the results to the database
save_dataframe_to_db(final_metadata_df, "newspapers", db_path="data_output/dwh_data.db", if_exists="replace")
save_dataframe_to_db(final_context_df, "context", db_path="data_output/dwh_data.db", if_exists="replace")

2025-02-22 18:00:08,620 - INFO - Data saved to table 'newspapers' in 'data_output/dwh_data.db' successfully.
2025-02-22 18:00:08,667 - INFO - Data saved to table 'context' in 'data_output/dwh_data.db' successfully.


In [23]:
# check the saved data
temp_data_head = read_table_as_dataframe("newspapers", "data_output/dwh_data.db").head()
temp_data_head

2025-02-22 18:00:38,270 - INFO - Data read from table 'newspapers' in 'data_output/dwh_data.db' successfully.


,newspaper,data_published,klima_mentions_count,newspaper_id
0,54books,2021-04-02,0,1
1,abendblatt,2021-04-02,1,2
2,vice,2025-01-24,0,3
3,tagesschaus,2021-04-24,2,4
4,tagesschaus,2025-01-24,4,5


In [25]:
temp_data_head = read_table_as_dataframe("context", "data_output/dwh_data.db").head()
temp_data_head

2025-02-22 18:01:22,118 - INFO - Data read from table 'context' in 'data_output/dwh_data.db' successfully.


,pre_context,post_context,prefix,suffix,newspaper_id
0,Was geht vor:,oder Mieterinteresse? Hamburg,,schutz,2
1,Adam. Biden zur,Jobmotor nicht Jobkiller,,politik,4
2,Kampf gegen den,schafft Jobs -,,wandel,4
3,Startseite Wissen Gesundheit,& Umwelt Forschung,,,5
4,Millionen Kinder betroffen,schränkt Schulbildung weltweit,,wandel,5
